In [ ]:

"""
transcript format:
{
    "video_id": "4pa9O5cr398",
    "timestamp": "2025-04-06T19:14:11.761808",
    "status": "success",
    "error_type": null,
    "error_message": null,
    "transcripts": [
        {
            "language": "English (auto-generated)",
            "language_code": "en",
            "is_generated": true,
            "data": [
                {
                    "text": "hello",
                    "start": 9.28,
                    "duration": 2.88
                },
                {
...
                {
                    "text": "you",
                    "start": 723.12,
                    "duration": 2.08
                }
            ]
        }
    ]
}
"""

In [ ]:
import pandas as pd
import os
import json
import re
import numpy as np
from tqdm import tqdm
from datetime import datetime
from typing import List, Dict, Any, Tuple
from collections import defaultdict

sponsor_dataset = r"C:\Users\caotr\Downloads\transcript_data\sponsorTimes.parquet"
video_info_dataset = r"C:\Users\caotr\Downloads\transcript_data\videoInfo.csv"
transcript_folder = r"C:\Users\caotr\Downloads\transcript_data\transcripts"

In [ ]:
def process_transcripts(transcript_folder, sponsor_dataset):
    # Đọc dữ liệu sponsor
    sponsor_df = pd.read_parquet(sponsor_dataset)
    
    # Tạo dictionary để lưu thông tin sponsor theo video_id
    sponsor_dict = defaultdict(list)
    for _, row in sponsor_df.iterrows():
        sponsor_dict[row['videoID']].append((row['startTime'], row['endTime']))
    
    all_segments = []
    
    # Đọc và xử lý từng file transcript
    for file_name in tqdm(os.listdir(transcript_folder)):
        if file_name.endswith('.json'):
            file_path = os.path.join(transcript_folder, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                transcript = json.load(f)
            
            video_id = transcript['video_id']
            
            # Kiểm tra xem transcript có dữ liệu không
            if transcript['status'] != 'success' or not transcript['transcripts']:
                continue
            
            # Lấy dữ liệu transcript
            transcript_data = None
            for t in transcript['transcripts']:
                if t['language_code'] == 'en':
                    transcript_data = t['data']
                    break
            
            if not transcript_data:
                continue
            
            # Xử lý từng đoạn trong transcript
            for segment in transcript_data:
                start_time = segment['start']
                end_time = start_time + segment['duration']
                text = segment['text']
                
                # Xác định xem đoạn này có nằm trong sponsor không
                is_sponsor = 0
                for sponsor_start, sponsor_end in sponsor_dict.get(video_id, []):
                    # Nếu đoạn này nằm chủ yếu trong khoảng sponsor
                    if (start_time >= sponsor_start and start_time < sponsor_end) or \
                       (end_time > sponsor_start and end_time <= sponsor_end) or \
                       (start_time <= sponsor_start and end_time >= sponsor_end):
                        is_sponsor = 1
                        break
                
                all_segments.append({
                    'video_id': video_id,
                    'start': start_time,
                    'end': end_time,
                    'text': text,
                    'is_sponsor': is_sponsor
                })
    
    # Tạo DataFrame từ tất cả các đoạn đã xử lý
    segments_df = pd.DataFrame(all_segments)
    return segments_df

# Gọi hàm xử lý
segments_df = process_transcripts(transcript_folder, sponsor_dataset)

# Lưu DataFrame để sử dụng sau này
segments_df.to_parquet('transcript_segments_labeled.parquet', index=False)